In [5]:
from dotenv import load_dotenv
import os
load_dotenv()
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def say_hello():
    response = client.chat.completions.create(
        model="gpt-5-nano",  # or another available model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Say hello to me in one short sentence."}
        ]
    )
    # Get the text of the assistant's reply
    return response.choices[0].message.content

if __name__ == "__main__":
    hello_message = say_hello()
    print(hello_message)


Hello there!


In [6]:
from newspaper import Article

urls = [
    "https://aeon.co/essays/how-the-fall-of-the-roman-empire-paved-the-road-to-modernity",
    "https://www.bbc.co.uk/history/ancient/romans/fallofrome_article_01.shtml",
]

def fetch_and_print(urls):
    for url in urls:
        try:
            article = Article(url)
            article.download()
            article.parse()

            print(f"----- Text content from: {url} -----\n")
            print(article.text[:2000])
            print("\n\n")

        except Exception as e:
            print(f"Error fetching {url}: {e}")

if __name__ == "__main__":
    fetch_and_print(urls)

----- Text content from: https://aeon.co/essays/how-the-fall-of-the-roman-empire-paved-the-road-to-modernity -----

For an empire that collapsed more than 1,500 years ago, ancient Rome maintains a powerful presence. About 1 billion people speak languages derived from Latin; Roman law shapes modern norms; and Roman architecture has been widely imitated. Christianity, which the empire embraced in its sunset years, remains the world’s largest religion. Yet all these enduring influences pale against Rome’s most important legacy: its fall. Had its empire not unravelled, or had it been replaced by a similarly overpowering successor, the world wouldn’t have become modern.

This isn’t the way that we ordinarily think about an event that has been lamented pretty much ever since it happened. In the late 18th century, in his monumental work The History of the Decline and Fall of the Roman Empire (1776-1788), the British historian Edward Gibbon called it ‘the greatest, perhaps, and most awful scen

In [10]:
import os
from newspaper import Article
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

urls = [
    "https://aeon.co/essays/how-the-fall-of-the-roman-empire-paved-the-road-to-modernity",
    "https://www.bbc.co.uk/history/ancient/romans/fallofrome_article_01.shtml",
]

def fetch_articles(urls):
    articles = []
    for url in urls:
        try:
            article = Article(url)
            article.download()
            article.parse()

            articles.append({
                "url": url,
                "text": article.text
            })

        except Exception as e:
            print(f"Error fetching {url}: {e}")

    return articles


def generate_podcast_script(articles):
    sources_text = ""
    for i, a in enumerate(articles, start=1):
        sources_text += f"\nSOURCE {i} ({a['url']}):\n{a['text']}\n"

    prompt = f"""
You are a podcast scriptwriter.

Task:
Using the sources below, write an ORIGINAL podcast script suitable for a
5–10 minute episode.

Guidelines:
- Target audience: curious general listeners
- Tone: clear, engaging, slightly narrative
- Do NOT copy sentences or paragraphs from the sources
- Rephrase ideas in your own words
- Structure:
  1. Hook (30–45 seconds)
  2. Context and background
  3. 3–5 key ideas with explanations
  4. Short recap
  5. Reflective closing question

Optional:
- Light host cues like [PAUSE], [MUSIC IN], [MUSIC OUT]

Sources:
{sources_text}
"""

    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": "You write engaging podcast scripts."},
            {"role": "user", "content": prompt},
        ]
    )

    return response.choices[0].message.content


if __name__ == "__main__":
    articles = fetch_articles(urls)
    script = generate_podcast_script(articles)

    print("\n===== GENERATED PODCAST SCRIPT =====\n")
    print(script)

    speech = client.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input=script[:4000]
)

    output_file = "podcast_episode.mp3"

    with open(output_file, "wb") as f:
        f.write(speech.read())

    print(f"Audio file created: {output_file}")



===== GENERATED PODCAST SCRIPT =====

[MUSIC IN — a light, reflective tune fades in]

Host: Imagine standing at the edge of a vast, once-mighty empire. The roads you’ve known fade into ruins, the coin you once used barely clinks in your pocket, and new rulers arrive with unfamiliar banners. It sounds like a catastrophe. But what if that very moment—the collapse—wasn’t the end of something, but the opening of something else entirely? A doorway through which a very different kind of world could begin to take shape. Welcome to Across the Ruins, where we ask how the fall of an empire helped spark a path to our modern world.

[PAUSE, MUSIC FADES]

Context and background

Host: We’re talking about the fall of Rome in the western half of its empire—a transition that didn’t just rewrite maps, but rewired how power, money, and ideas circulated. By the late 5th century, central Rome’s grip loosened. Germanic kingdoms emerged on old soil, and the “Roman state” as a single, centralized machine ga

In [20]:
import os
import re
import shutil
import subprocess
from pathlib import Path

from dotenv import load_dotenv
from newspaper import Article
from openai import OpenAI

# -----------------------------
# Config
# -----------------------------
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

urls = [
    "https://aeon.co/essays/how-the-fall-of-the-roman-empire-paved-the-road-to-modernity",
    "https://www.bbc.co.uk/history/ancient/romans/fallofrome_article_01.shtml",
    "https://www.khanacademy.org/humanities/whp-origins/x23c41635548726c4:regional-webs/x23c41635548726c4:why-do-empires-collapse-5-2/a/article-the-fall-of-rome"
]

OUTPUT_DIR = Path("podcast_output")
CHUNKS_DIR = OUTPUT_DIR / "chunks"
FINAL_AUDIO = OUTPUT_DIR / "podcast_episode_final.mp3"

TTS_MODEL = "tts-1"
TTS_VOICE = "onyx"

# Keep headroom for TTS limits
MAX_CHARS_PER_CHUNK = 3500


# -----------------------------
# Fetch + Script generation
# -----------------------------
def fetch_articles(urls):
    articles = []
    for url in urls:
        try:
            article = Article(url)
            article.download()
            article.parse()
            articles.append({"url": url, "text": article.text})
        except Exception as e:
            print(f"Error fetching {url}: {e}")
    return articles


def generate_podcast_script(articles):
    sources_text = ""
    for i, a in enumerate(articles, start=1):
        sources_text += f"\nSOURCE {i} ({a['url']}):\n{a['text']}\n"

    prompt = f"""

You are a podcast scriptwriter and narrator.
Task: Using the sources below, write an ORIGINAL podcast script about the fall of the Roman Empire, suitable for a 8–10 minute episode.
Guidelines for the script:
Audience: curious general listeners
Tone: clear, engaging, slightly narrative, cinematic
Do NOT copy sentences or paragraphs; rephrase ideas in your own words
Highlight lessons from history – show how events from the past may provide insights into today
Structure:
Hook (30–45 seconds) – grab attention with a dramatic scene and the idea that history repeats itself; for example:
Introduce Rome as one of the most advanced and significant civilizations ever
Emphasize that understanding its decline can give clues about patterns in our world today
Pose a reflective question to the listener: “Could the lessons of ancient Rome help us understand the challenges of our own time?”
Context and background – explain the empire’s peak, structure, significance, and key players
3–5 key ideas – focus on the main causes and consequences of Rome’s fall, explained clearly and with storytelling flair
Short recap – summarize the main points
Reflective closing question – tie back to learning from history and its relevance today
Narration style:
Speak in a natural, clear, and engaging voice
Use expressive tone, pacing, and pauses to emphasize dramatic moments
Ensure the listener can follow the story easily while staying engaged
End result: A script ready for a podcast episode that explains the fall of the Roman Empire, its key players, and the lessons we can learn today.

Sources:
{sources_text}
"""

    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": "You write engaging podcast scripts."},
            {"role": "user", "content": prompt},
        ],
    )
    return response.choices[0].message.content


# -----------------------------
# Chunking
# -----------------------------
def normalize_whitespace(text: str) -> str:
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()


def chunk_text(text: str, max_chars: int = MAX_CHARS_PER_CHUNK) -> list[str]:
    """
    Chunk by paragraphs, then by sentences if needed.
    Avoids cutting mid-sentence when possible.
    Hard-slices only as a last resort.
    """
    text = normalize_whitespace(text)
    paragraphs = [p.strip() for p in text.split("\n\n") if p.strip()]

    chunks = []
    current = ""

    def flush():
        nonlocal current
        if current.strip():
            chunks.append(current.strip())
        current = ""

    sentence_split = re.compile(r"(?<=[.!?])\s+")

    for p in paragraphs:
        parts = [p]
        if len(p) > max_chars:
            parts = sentence_split.split(p)

        for part in parts:
            part = part.strip()
            if not part:
                continue

            candidate = part if not current else current + "\n\n" + part

            if len(candidate) <= max_chars:
                current = candidate
            else:
                flush()

                if len(part) > max_chars:
                    for i in range(0, len(part), max_chars):
                        piece = part[i : i + max_chars].strip()
                        if piece:
                            chunks.append(piece)
                else:
                    current = part

    flush()
    return chunks


# -----------------------------
# TTS
# -----------------------------
def tts_to_mp3(text: str, out_path: Path):
    speech = client.audio.speech.create(
        model=TTS_MODEL,
        voice=TTS_VOICE,
        input=text,
    )
    out_path.parent.mkdir(parents=True, exist_ok=True)
    with open(out_path, "wb") as f:
        f.write(speech.read())


# -----------------------------
# Audio concat (absolute paths)
# -----------------------------
def ffmpeg_available() -> bool:
    return shutil.which("ffmpeg") is not None


def _ffmpeg_concat_escape(path: Path) -> str:
    # concat demuxer lines are: file '...'
    # use absolute posix path + escape single quotes
    p = path.resolve().as_posix()
    return p.replace("'", r"'\''")


def concat_mp3_ffmpeg(mp3_files: list[Path], out_file: Path):
    """
    Robust concat:
      1) Try stream-copy concat (fast, may fail for MP3 chunks).
      2) Retry with re-encode (reliable).
    Uses absolute paths in concat_list to avoid path duplication issues.
    """
    out_file.parent.mkdir(parents=True, exist_ok=True)
    list_file = out_file.parent / "concat_list.txt"

    # Ensure all files exist before calling ffmpeg
    missing = [p for p in mp3_files if not p.exists()]
    if missing:
        raise FileNotFoundError(f"Missing chunk files: {missing}")

    with open(list_file, "w", encoding="utf-8") as f:
        for p in mp3_files:
            f.write(f"file '{_ffmpeg_concat_escape(p)}'\n")

    def run(cmd: list[str]):
        res = subprocess.run(cmd, capture_output=True, text=True)
        if res.returncode != 0:
            raise RuntimeError(
                "ffmpeg concat failed.\n"
                f"CMD: {' '.join(cmd)}\n\n"
                f"STDOUT:\n{res.stdout}\n\nSTDERR:\n{res.stderr}"
            )
        return res

    # 1) Stream-copy
    try:
        run([
            "ffmpeg", "-y",
            "-f", "concat", "-safe", "0",
            "-i", str(list_file),
            "-c", "copy",
            str(out_file),
        ])
        return
    except RuntimeError as e:
        print(str(e))
        print("\nRetrying with re-encode (more reliable)...\n")

    # 2) Re-encode
    run([
        "ffmpeg", "-y",
        "-f", "concat", "-safe", "0",
        "-i", str(list_file),
        "-vn",
        "-c:a", "libmp3lame",
        "-q:a", "2",
        "-ar", "44100",
        "-ac", "2",
        str(out_file),
    ])


def concat_mp3_pydub(mp3_files: list[Path], out_file: Path):
    from pydub import AudioSegment  # pip install pydub

    combined = AudioSegment.empty()
    for p in mp3_files:
        combined += AudioSegment.from_file(p, format="mp3")

    out_file.parent.mkdir(parents=True, exist_ok=True)
    combined.export(out_file, format="mp3")


# -----------------------------
# Main
# -----------------------------
def main():
    OUTPUT_DIR.mkdir(exist_ok=True)
    CHUNKS_DIR.mkdir(parents=True, exist_ok=True)

    articles = fetch_articles(urls)
    if not articles:
        raise RuntimeError("No articles fetched. Check URLs / connectivity.")

    script = generate_podcast_script(articles)

    print("\n===== GENERATED PODCAST SCRIPT (preview) =====\n")
    print(script + ("\n...\n" if len(script) > 2000 else ""))

    chunks = chunk_text(script, max_chars=MAX_CHARS_PER_CHUNK)
    print(f"\nChunked script into {len(chunks)} parts (<= {MAX_CHARS_PER_CHUNK} chars each).")


    mp3_files = []
    for idx, chunk in enumerate(chunks, start=1):
        mp3_path = CHUNKS_DIR / f"chunk_{idx:02d}.mp3"
        print(f"Generating audio for chunk {idx}/{len(chunks)} -> {mp3_path}")
        tts_to_mp3(chunk, mp3_path)
        mp3_files.append(mp3_path)

    print("\nCombining chunks into final audio...")

    if ffmpeg_available():
        concat_mp3_ffmpeg(mp3_files, FINAL_AUDIO)
    else:
        print("ffmpeg not found in PATH. Trying pydub fallback...")
        concat_mp3_pydub(mp3_files, FINAL_AUDIO)

    print(f"\nDone. Final audio file created: {FINAL_AUDIO.as_posix()}")

if __name__ == "__main__":
    main()



===== GENERATED PODCAST SCRIPT (preview) =====

Hook

SFX: a faint wind across stone, then the creak of a timber door. A distant hammer on an anvil, markets waking, horses neighing.

NARRATOR: Imagine a city built on stone and speech: Rome at its zenith, a tapestry of law, roads, temples, and a language that would echo for centuries. It’s not just a museum showpiece. It’s a living machine of power, culture, and engineering that once stitched a world together. And then—dramatically, almost overnight—the machine falters. Armies shift, money vanishes from the mint, and a city that once sent legions across continents becomes a shadow of its former self. What went wrong? And more hauntingly: if a superpower can fall, what patterns in our own world might be repeating themselves right now?

Rome was not merely a city or a collection of rulers. It was one of humanity’s greatest experiments in scale, law, and unity. Its fall didn’t just mark the end of an empire; it seeded a long, uneven ascen

In [ ]:
import os
import re
import shutil
import subprocess
from pathlib import Path

from dotenv import load_dotenv
from openai import OpenAI

# -----------------------------
# Config
# -----------------------------
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# TTS model: set this to what your account supports.
# Try: "gpt-4o-mini-tts" or fallback to "tts-1"
TTS_MODEL = "gpt-4o-mini-tts"
HOST_VOICE = "alloy"
COHOST_VOICE = "cedar"

OUTPUT_DIR = Path("tts_two_voice_test")
SEG_DIR = OUTPUT_DIR / "segments"
FINAL_AUDIO = OUTPUT_DIR / "two_voice_test_final.mp3"

PAUSE_MS_BETWEEN_TURNS = 450  # pacing pause


# -----------------------------
# Short test script (two speakers)
# -----------------------------
TEST_SCRIPT = """
[HOST]
Hey! Welcome back. Quick experiment: can AI do a natural two-voice podcast?

[COHOST]
Let’s find out. I’ll ask questions, you keep us on track.

[HOST]
Deal. First topic: why we should keep sentences short for text to speech.

[COHOST]
Because long sentences sound like someone reading an essay out loud.

[HOST]
Exactly. Okay, let’s render this and stitch it together.
""".strip()


# -----------------------------
# Helpers
# -----------------------------
def ffmpeg_available() -> bool:
    return shutil.which("ffmpeg") is not None


def run(cmd: list[str]) -> None:
    res = subprocess.run(cmd, capture_output=True, text=True)
    if res.returncode != 0:
        raise RuntimeError(
            "Command failed.\n"
            f"CMD: {' '.join(cmd)}\n\n"
            f"STDOUT:\n{res.stdout}\n\nSTDERR:\n{res.stderr}"
        )


def parse_dialogue(script: str):
    """
    Parses blocks like:
      [HOST]
      text...
      [COHOST]
      text...
    Returns: list of {"speaker": "...", "text": "..."}
    """
    lines = [ln.rstrip() for ln in script.splitlines()]
    blocks = []
    current_speaker = None
    current_text = []

    tag_re = re.compile(r"^\[(HOST|COHOST)\]\s*$")

    def flush():
        nonlocal current_speaker, current_text
        if current_speaker and any(t.strip() for t in current_text):
            blocks.append(
                {"speaker": current_speaker, "text": "\n".join(current_text).strip()}
            )
        current_text = []

    for ln in lines:
        m = tag_re.match(ln.strip())
        if m:
            flush()
            current_speaker = m.group(1)
        else:
            if current_speaker is not None:
                current_text.append(ln)

    flush()
    return blocks


def tts_segment(text: str, voice: str, out_path: Path):
    """
    Generates one MP3 segment for a single speaker turn.
    """
    out_path.parent.mkdir(parents=True, exist_ok=True)

    speech = client.audio.speech.create(
        model=TTS_MODEL,
        voice=voice,
        input=text,
    )

    with open(out_path, "wb") as f:
        f.write(speech.read())


def make_silence_mp3(duration_ms: int, out_path: Path):
    """
    Creates a silent MP3 using ffmpeg.
    """
    out_path.parent.mkdir(parents=True, exist_ok=True)
    seconds = max(duration_ms / 1000.0, 0.01)

    run([
        "ffmpeg", "-y",
        "-f", "lavfi",
        "-i", f"anullsrc=r=44100:cl=stereo",
        "-t", f"{seconds:.3f}",
        "-c:a", "libmp3lame",
        "-q:a", "2",
        str(out_path),
    ])


def concat_mp3_with_reencode(mp3_files: list[Path], out_file: Path):
    """
    Concats MP3s robustly using ffmpeg concat demuxer and re-encodes.
    Uses absolute paths to avoid path issues.
    """
    out_file.parent.mkdir(parents=True, exist_ok=True)
    list_file = out_file.parent / "concat_list.txt"

    for p in mp3_files:
        if not p.exists():
            raise FileNotFoundError(f"Missing file: {p}")

    def esc(p: Path) -> str:
        return p.resolve().as_posix().replace("'", r"'\''")

    with open(list_file, "w", encoding="utf-8") as f:
        for p in mp3_files:
            f.write(f"file '{esc(p)}'\n")

    run([
        "ffmpeg", "-y",
        "-f", "concat", "-safe", "0",
        "-i", str(list_file),
        "-vn",
        "-c:a", "libmp3lame",
        "-q:a", "2",
        "-ar", "44100",
        "-ac", "2",
        str(out_file),
    ])


# -----------------------------
# Main
# -----------------------------
def main():
    if not ffmpeg_available():
        raise RuntimeError("ffmpeg not found. Install ffmpeg and try again.")

    OUTPUT_DIR.mkdir(exist_ok=True)
    SEG_DIR.mkdir(parents=True, exist_ok=True)

    turns = parse_dialogue(TEST_SCRIPT)
    if not turns:
        raise RuntimeError("No dialogue turns parsed. Check your [HOST]/[COHOST] tags.")

    voice_map = {"HOST": HOST_VOICE, "COHOST": COHOST_VOICE}

    mp3_sequence = []
    silence_path = SEG_DIR / f"pause_{PAUSE_MS_BETWEEN_TURNS}ms.mp3"
    make_silence_mp3(PAUSE_MS_BETWEEN_TURNS, silence_path)

    print(f"Generating {len(turns)} segments with two voices using model={TTS_MODEL}...")

    for i, turn in enumerate(turns, start=1):
        speaker = turn["speaker"]
        text = turn["text"]

        out_mp3 = SEG_DIR / f"seg_{i:02d}_{speaker.lower()}.mp3"
        tts_segment(text=text, voice=voice_map[speaker], out_path=out_mp3)
        mp3_sequence.append(out_mp3)

        # Add pacing pause after each turn except the last
        if i != len(turns):
            mp3_sequence.append(silence_path)

        print(f"  {i:02d}. {speaker} -> {out_mp3.name}")

    print("Concatenating into final audio...")
    concat_mp3_with_reencode(mp3_sequence, FINAL_AUDIO)

    print(f"Done: {FINAL_AUDIO}")


if __name__ == "__main__":
    main()

Generating 5 segments with two voices using model=gpt-4o-mini-tts...
  01. HOST -> seg_01_host.mp3
  02. COHOST -> seg_02_cohost.mp3
  03. HOST -> seg_03_host.mp3
  04. COHOST -> seg_04_cohost.mp3
  05. HOST -> seg_05_host.mp3
Concatenating into final audio...
Done: tts_two_voice_test/two_voice_test_final.mp3
